In [1]:
import numpy as np
import mujoco
import mediapy
import matplotlib.pyplot as plt
import time
import os

In [2]:
video_index = 0
def load_model_and_reset_data_from_xml(xml_string):
    mj_model = mujoco.MjModel.from_xml_string(xml_string)
    mj_data = mujoco.MjData(mj_model)

    mujoco.mj_resetData(mj_model, mj_data)
    return mj_data, mj_model
    # mj_data.joint('root').qvel = initial_qvel
def render(mj_data, mj_model, framerate=30, play_speed=1, num_frames=30 * 10, save_file_name = None):
    global video_index
    mj_renderer = mujoco.Renderer(mj_model)

    frames = []
    mujoco.mj_forward(mj_model, mj_data)
    for idx in range(num_frames):
        while mj_data.time * framerate / play_speed < idx:
            mujoco.mj_step(mj_model, mj_data)

        # mj_renderer.update_scene(mj_data, camera="fixed")
        mj_renderer.update_scene(mj_data, camera=0)
        frames.append(mj_renderer.render())

    mj_renderer.close()
    mediapy.show_video(frames, fps=framerate)
    video_index += 1
    if save_file_name is not None:
        path = os.path.join("result_videos", f"part3_{video_index:03d}_{save_file_name}")
        mediapy.write_video(path, frames, fps=framerate)

# equality

In [3]:
stiffness1 = 10
damping1 = 0.5
hinge_stiffness1 = 0.1
hinge_damping1 = 0.0

stiffness2 = 10
damping2 = 0.5
hinge_stiffness2 = 0.1
hinge_damping2 = 0.0

stiffness3 = 10
damping3 = 0.5
hinge_stiffness3 = 0.1
hinge_damping3 = 0.0

mjcf_xml_weld = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -2 -0.5" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.03">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <!-- First pendulum 1 (double pendulum) -->
    <body name="pendulum1" pos="0 0 0">
      <joint name="joint1"/>
      <geom name="rod1" fromto="0 0 0 0 0 -0.5" rgba="1 1 0 1" mass = "1"/>
      <body name="pendulum1_lower" pos="0 0 -0.5">
        <joint name="joint1_lower"/>
        <geom name="rod1_lower" fromto="0 0 0 0 0 -0.5" rgba="1 0.5 0 1"/>
        <body name="weight1" pos="0 0 -0.5">
            <geom type="sphere" size=".05" rgba="1 0 0 1" mass="1"/>
            <!-- Exo -->
            <body name="weight1_exo" pos="0 0 0">
                <geom type="box" size=".06 .06 .06" rgba="1 0 1 1" mass="0.1"/>
                <geom type="capsule" size="0.003" rgba="1 0 0 1" mass="0" fromto="0 0 0 0.25 0 0"/>
                <geom type="capsule" size="0.003" rgba="0 1 0 1" mass="0" fromto="0 0 0 0 0.25 0"/>
                <geom type="capsule" size="0.003" rgba="0 0 1 1" mass="0" fromto="0 0 0 0 0 0.25"/>
                <joint type="slide" axis="0 0 1" stiffness="{stiffness1}" damping="{damping1}" range="-0.01 0.01" ref="0"/>
                <joint type="slide" axis="0 1 0" stiffness="{stiffness1}" damping="{damping1}" range="-0.01 0.01" ref="0"/>
                <joint type="slide" axis="1 0 0" stiffness="{stiffness1}" damping="{damping1}" range="-0.01 0.01" ref="0"/>
                <joint name="springdamperrz" axis="0 0 1" stiffness="{hinge_stiffness1}" damping="{hinge_damping1}" ref="0"/>
                <joint name="springdamperry" axis="0 1 0" stiffness="{hinge_stiffness1}" damping="{hinge_damping1}" ref="0"/>
                <joint name="springdamperrx" axis="1 0 0" stiffness="{hinge_stiffness1}" damping="{hinge_damping1}" ref="0"/>
                <body name = "weight2_child" pos="0 0 0.25">
                    <geom name="rod2" type="box" size="0.06 0.06 0.06" pos="0 0 0" rgba="0 1 0 1" mass="0.1"/>
                    <geom type="capsule" size="0.003" rgba="1 0 0 1" mass="0" fromto="0 0 0 0.25 0 0"/>
                    <geom type="capsule" size="0.003" rgba="0 1 0 1" mass="0" fromto="0 0 0 0 0.25 0"/>
                    <geom type="capsule" size="0.003" rgba="0 0 1 1" mass="0" fromto="0 0 0 0 0 0.25"/>

                    <joint type="slide" axis="0 1 0" stiffness="{stiffness2}" damping="{damping2}" range="-0.01 0.01" ref="0"/>
                    <joint type="slide" axis="1 0 0" stiffness="{stiffness2}" damping="{damping2}" range="-0.01 0.01" ref="0"/>
                    <joint name="springdamperrz2" axis="0 0 1" stiffness="{hinge_stiffness2}" damping="{hinge_damping2}" ref="0"/>
                    <joint name="springdamperry2" axis="0 1 0" stiffness="{hinge_stiffness2}" damping="{hinge_damping2}" ref="0"/>
                    <joint name="springdamperrx2" axis="1 0 0" stiffness="{hinge_stiffness2}" damping="{hinge_damping2}" ref="0"/>

                    <body name="weight2_child_constraint" pos="0 0 0">
                        <joint type="slide" axis="0 0 1" stiffness="{stiffness3}" damping="{damping3}" range="-0.1 0.1" ref="0"/>
                        <joint type="slide" axis="0 1 0" stiffness="{stiffness3}" damping="{damping3}" range="-0.1 0.1" ref="0"/>
                        <joint type="slide" axis="1 0 0" stiffness="{stiffness3}" damping="{damping3}" range="-0.03 0.03" ref="0"/>
                        <joint name="springdamperrz3" axis="0 0 1" stiffness="{hinge_stiffness3}" damping="{hinge_damping3}" ref="0"/>
                        <joint name="springdamperry3" axis="0 1 0" stiffness="{hinge_stiffness3}" damping="{hinge_damping3}" ref="0"/>
                        <joint name="springdamperrx3" axis="1 0 0" stiffness="{hinge_stiffness3}" damping="{hinge_damping3}" ref="0"/>
                        <!--mass should not be super small (underflow?)-->
                        <geom type="box" size=".03 .03 .03" rgba="0 1 1 1" mass="0.001"/>
                    </body>
                </body>
            </body>
        </body>
      </body>
    </body>
    
    <!-- pendulum 2 -->
    
  </worldbody>
  
  <!-- Equality constraints section using weld -->
  <equality>
    <!-- Weld the two pendulum weights together with a fixed relative position -->
    <weld body1="pendulum1_lower" body2="weight2_child_constraint" solimp="0.9 0.95 0.001" solref="0.02 1"/>
  </equality>
</mujoco>
"""

mj_data_weld, mj_model_weld = load_model_and_reset_data_from_xml(mjcf_xml_weld)

# Set initial velocity to the first pendulum
mj_data_weld.joint('joint1').qpos = 0.3
# mj_data_weld.joint('springdamperrx').qpos = 0.3
# mj_data_weld.joint('springdamperry').qpos = 0.3
mj_data_weld.joint('springdamperrz').qpos = 1
mj_data_weld.joint('springdamperrz2').qpos = 1
mj_data_weld.joint('springdamperrz3').qpos = 1

# Render the simulation with weld constraint
render(mj_data_weld, mj_model_weld, play_speed=1, num_frames=30 * 5, save_file_name="weld_equality_constraint_with_springdamper.mp4")

This browser does not support the video tag.
